In [1]:
import igraph as ig
import numpy as np
import louvain
import time

In [2]:
print("Construct the graph from the txt file (edgelist)")

input = np.loadtxt("Data/ElistPowerEU.txt", dtype='i', delimiter=' ');
dim = input.shape;
N = input.max();
g = ig.Graph();
g.add_vertices(N);
for i in range(0, dim[0]):
    g.add_edges([(input[i,0]-1,input[i,1]-1)]);
ig.summary(g)

Construct the graph from the txt file (edgelist)
IGRAPH U--- 2783 3601 -- 


In [3]:
print("Largest connected component")
largest = g.clusters().giant();
ig.summary(largest)

Largest connected component
IGRAPH U--- 2712 3580 -- 


In [5]:
print("Running Louvain 1 times in C++")
start_time = time.time()

part = louvain.find_partition(largest, louvain.ModularityVertexPartition);

        
print("--- %s seconds ---" % (time.time() - start_time))

print("Modularity :",part.modularity)
print("Sizes of the groups:", part.sizes())

Running Louvain 1 times in C++
--- 0.034647464752197266 seconds ---
Modularity : 0.9191233497706065
Sizes of the groups: [155, 138, 134, 130, 126, 124, 117, 110, 104, 96, 94, 92, 92, 91, 90, 90, 87, 84, 78, 73, 71, 64, 63, 58, 58, 56, 53, 50, 47, 47, 20, 20]


In [9]:
ig.summary(largest);
print("Running Louvain 100 times in C++")
start_time = time.time()

m_best =0;
part_best = louvain.find_partition(largest, louvain.ModularityVertexPartition);


for n in range(100):
    part = louvain.find_partition(largest, louvain.ModularityVertexPartition);
    m = part.modularity;
    if m_best<m:
        m_best = m;
        part_best = part;
        

print("--- %s seconds ---" % (time.time() - start_time))

print("Modularity :",part_best.modularity)
print("Sizes of the groups:", part_best.sizes())

IGRAPH U--- 2712 3580 -- 
Running Louvain 100 times in C++
--- 3.5878243446350098 seconds ---
Modularity : 0.9216666536624947
Sizes of the groups: [155, 140, 133, 126, 115, 114, 105, 103, 103, 103, 98, 96, 96, 93, 88, 87, 82, 79, 76, 75, 72, 69, 69, 67, 64, 60, 58, 53, 48, 45, 20, 20]


In [10]:
communities_Louvain = part_best.membership;
Q = part_best.modularity;
import numpy, scipy.io

scipy.io.savemat('Data/Louvain_PowerEU.mat', mdict={'communities_Louvain': communities_Louvain})

In [5]:
len(part_best)

86

In [5]:
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix

membership =part_best.membership;
H = lil_matrix((part_best.n,len(part_best.sizes())));
i = 0;
for n in membership:
    H[i,n] =1.;
    i = i+1;
    
e_list = largest.get_edgelist();
e_list = np.array(e_list);

A = lil_matrix((part_best.n,part_best.n), dtype=np.int8);

for n in range(len(e_list)):
    A[e_list[n,0],e_list[n,1]] = 1;

A = A+lil_matrix.transpose(A);
deg = A.sum(0)


L_n = lil_matrix((part_best.n,part_best.n));

for n in range(len(e_list)):
    L_n[e_list[n,0],e_list[n,1]] = -1./np.sqrt(deg[0,e_list[n,0]]*deg[0,e_list[n,1]]);
    
L_n = L_n+lil_matrix.transpose(L_n);


for a in range(part_best.n):
    L_n[a,a] = 1.;
            

T =(lil_matrix.transpose(H)*L_n*H);
d = np.array(T.diagonal());
n_cut = np.sum(d);
print('Normalized Cut')
print(n_cut)

/home/fanuel/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Normalized Cut
348.021857911
